## CSC14005 – MACHINE LEARNING
## FINAL PROJECT

### Students
- Le Hoang Sang - 21127158
- Tran Minh Quang - 21127

### Prepare libs 

In [1]:
import fashion_mnist.utils.mnist_reader as mnist_reader
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
import time
import numpy as np

X_train_org, y_train_org = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='train')
X_test_org, y_test_org = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='t10k')

In [2]:
# normalize data
X_train_org = X_train_org / 255
X_test_org = X_test_org / 255
X_train_org.shape

(60000, 784)

In [3]:
y_train_org.shape

(60000,)

### Using Linear Kernel

In [14]:
#linear model

# model_linear = SVC(kernel='linear', C = 0.5)
# model_linear.fit(X_train, y_train)

# # predict
# y_pred = model_linear.predict(X_test)

# print(classification_report(y_test, y_pred))

# ----------------------------------------------

# Tạo danh sách các giá trị C bạn muốn thử nghiệm
C_values = [0.1, 0.5, 1.0, 2.0, 5.0]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X_train_org, y_train_org, test_size=0.2, random_state=42)

for C in C_values:
    # Bắt đầu đo thời gian
    start_time = time.time()

    # Tạo và huấn luyện mô hình với giá trị C hiện tại
    model_linear = SVC(kernel='linear', C=C)
    model_linear.fit(X_train, y_train)

    # Đo thời gian huấn luyện
    training_time = time.time() - start_time

    # Tính độ lỗi trên tập huấn luyện
    train_error = 1 - model_linear.score(X_train, y_train)

    # Tính độ lỗi trên tập validation sử dụng cross-validation
    val_error = 1 - model_linear.score(X_val, y_val)

    # In ra kết quả
    print(f"C={C} - Training Error: {train_error:.4f}, Validation Error: {val_error:.4f}, Training Time: {training_time:.2f} seconds")

C=0.1 - Training Error: 0.1095, Validation Error: 0.1354, Training Time: 176.03 seconds
C=0.5 - Training Error: 0.0970, Validation Error: 0.1422, Training Time: 192.04 seconds
C=1.0 - Training Error: 0.0930, Validation Error: 0.1462, Training Time: 203.29 seconds
C=2.0 - Training Error: 0.0886, Validation Error: 0.1510, Training Time: 229.24 seconds
C=5.0 - Training Error: 0.0845, Validation Error: 0.1549, Training Time: 304.40 seconds


### Comment results

### Using RBF Kernel

In [5]:
# RBF model

# model_rbf = SVC(kernel='rbf', C = 0.5, gamma = 'auto')
# model_rbf.fit(X_train, y_train)

# # predict
# y_pred = model_rbf.predict(X_test)

# print(classification_report(y_test, y_pred))

# ------------------------------------------------

# Tạo danh sách các giá trị C và gamma bạn muốn thử nghiệm
C_values = [0.1, 0.5, 1.0, 2.0, 5.0]
gamma_values = ['auto', 0.1, 0.5, 1.0, 2.0]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X_train_org, y_train_org, test_size=0.2, random_state=42)

for C in C_values:
    for gamma in gamma_values:
        # Bắt đầu đo thời gian
        start_time = time.time()

        # Tạo và huấn luyện mô hình với giá trị C và gamma hiện tại
        model_rbf = SVC(kernel='rbf', C=C, gamma=gamma)
        model_rbf.fit(X_train, y_train)

        # Đo thời gian huấn luyện
        training_time = time.time() - start_time

        # Tính độ lỗi trên tập huấn luyện
        train_error = 1 - model_rbf.score(X_train, y_train)

        # Tính độ lỗi trên tập validation sử dụng cross-validation
        val_error = 1 - model_rbf.score(X_val, y_val)

        # In ra kết quả
        print(f"C={C}, gamma={gamma} - Training Error: {train_error:.4f}, Validation Error: {val_error:.4f}, Training Time: {training_time:.2f} seconds")

C=0.1, gamma=auto - Training Error: 0.2010, Validation Error: 0.2063, Training Time: 519.34 seconds
C=0.1, gamma=0.1 - Training Error: 0.2408, Validation Error: 0.2776, Training Time: 1253.40 seconds
C=0.1, gamma=0.5 - Training Error: 0.8969, Validation Error: 0.9023, Training Time: 2245.79 seconds


### Comment results

### Review SVM

### References